In [1]:
import pandas as pd
import bz2
import json
import re
from tqdm import tqdm

In [2]:
from pymystem3 import Mystem
m = Mystem()
ru_words = re.compile("[А-Яа-я^]+")

In [18]:
def words_only(text):
    return " ".join(ru_words.findall(text))


def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return ""
    
def preprocess(text):
    try:
        return lemmatize(words_only(text.lower()))
    except:
        None

def fast_preprocess(x):
    return preprocess("^".split(x)).join('^')

In [4]:
split_regex = re.compile(r'([А-ЯA-Z]((т.п.|т.д.|пр.|г.|пл.)|[^?!.\(]|\([^\)]*\))*[.?!])')
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if len(resp['text'].split()) > 0:
            responses.append(resp['text'])

201030it [01:04, 3116.71it/s]


In [5]:
text = ' '.join(responses)

In [6]:
sentences = []
for i,sent in enumerate(split_regex.findall(text)):
                sentences.append(sent[0])

In [7]:
sentences[:5]

['Здравствуйте!',
 'Хотелось бы выразить благодарность банку ВТБ24 г.Воронеж пл. Ленина и непосредственно специалисту по ипотечному кредитованию Мещеряковой Юлии Анатольевне.',
 'Это уже моя вторая ипотека в этом банке и у этого специалиста.',
 'Вопросов с подачей документов не было, быстрое принятие решения об одобрении всей необходимой суммы и никаких проблем с выдачей кредита.',
 'Удобно, что никуда не нужно ходить и со сторонними организациями не нужно договариваться.']

In [8]:
# lemmas = preprocess('^'.join(sentences)).split('^')
len(sentences)

3167222

In [35]:
def list_parts(lst, n):
    for x in range(0, len(lst), n):
        e_c = lst[x : n + x]
        yield e_c

In [31]:
lemsentences = []
for i in tqdm(list(list_parts(sentences, 100000))):
    lemmas = preprocess('^'.join(i)).split('^')
    lemsentences.extend(lemmas)

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [34:38<00:00, 64.95s/it]


In [33]:
# with open('lemsentences.txt', 'w') as file:  
#     for i in lemsentences:
#         file.write('%s\n' % i)

In [34]:
# ls = []

# # откроем файл и считаем его содержимое в список
# with open('lemsentences.txt', 'r') as file:
#     for line in file:
#         # удалим заключительный символ перехода строки
#         currentPlace = line[:-1]
#         # добавим элемент в конец списка
#         ls.append(currentPlace)
        
# print(len(ls), len(lemsentences))
# ls[:2]

3167237 3167237


['здравствовать ',
 'хотеться бы выражать благодарность банка втб г воронеж пл ленин и непосредственно специалист по ипотечный кредитование мещерякова юлия анатольевна ']

In [46]:
lslist = []
for i in lemsentences:
    lslist.append(i.split())

In [48]:
%time 
from gensim.models import word2vec
model = word2vec.Word2Vec(lslist, workers=4, min_count=10, window=10, sample=1e-3)

Wall time: 0 ns


In [51]:
print(len(model.wv.index_to_key))

24850


In [59]:
print(model.wv.most_similar(positive=["сбербанк", "карта"], negative=["клиент"], topn=1))
print(model.wv.most_similar(positive=["карта"], negative=["деньги"], topn=1))
print(model.wv.most_similar("банкомат", topn=3))
print(model.wv.doesnt_match("сбербанк втб альфабанк деньги".split()))

[('карточка', 0.5506956577301025)]
[('малина', 0.5217198133468628)]
[('терминал', 0.8411943912506104), ('атма', 0.8104454874992371), ('аппарат', 0.7659865021705627)]
деньги


In [124]:
print(model.wv.most_similar(positive=["сбербанк"], negative=["хамство"], topn=1))
print(model.wv.most_similar(positive=["сбербанк", "позитив"], topn=1))
print(model.wv.most_similar(positive=["банк"], negative=["клиент"], topn=1))
print(model.wv.most_similar(positive=["клиент"], negative=["проблема"], topn=1))
print(model.wv.most_similar(positive=["клиент", "кредит"], topn=1))
print(model.wv.most_similar(positive=["клиент", "вклад"], topn=1))
print(model.wv.most_similar(positive=["клиент", "долг"], topn=1))
print(model.wv.most_similar(positive=["коллектор"], negative=["работа"], topn=1))
print(model.wv.most_similar(positive=["работа"], negative=["деньги"], topn=1))

[('мкб', 0.5575763583183289)]
[('сбер', 0.6475073099136353)]
[('банка', 0.4777216911315918)]
[('вкладчик', 0.5421962738037109)]
[('заемщик', 0.6504892706871033)]
[('вкладчик', 0.7955532073974609)]
[('должник', 0.6807858347892761)]
[('агентство', 0.5696046948432922)]
[('общение', 0.4926299750804901)]


In [130]:
from nltk import FreqDist
from sklearn.manifold import TSNE

top_words = []


fd = FreqDist()
for s in tqdm(lslist):
    fd.update(s)

for w in fd.most_common(100):
    top_words.append(w[0])
    
print(top_words[:50:])


['в', 'я', 'и', 'не', 'на', 'что', 'банк', 'с', 'быть', 'по', 'карта', 'то', 'это', 'а', 'мой', 'как', 'но', 'этот', 'за', 'у', 'деньги', 'день', 'кредит', 'о', 'вы', 'так', 'весь', 'который', 'к', 'клиент', 'сотрудник', 'они', 'отделение', 'мочь', 'счет', 'от', 'же', 'она', 'свой', 'для', 'сказать', 'такой', 'все', 'из', 'уже', 'сумма', 'он', 'через', 'получать', 'вопрос']


In [135]:
top_words_vec = model.wv.__getitem__(top_words)
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [139]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...